# Sample testing for DEM & slopes

This is basically a sandbox. By playing with smaller area, eg, a single tile of TMS zoom 10, we can get accurate comparison of approaches.

* The `cut_extent` command will extract from an existing DEM.
* The `slope` command converts to mbtile.

See [gdal_slope_util.py](../src/gdal_slope_util.py)

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.curdir)))

In [2]:
from src.bbox import bbmontblancz10
from src.gdal_slope_util import cut_extent, make_ovr, slope_mbt, make_overviews, mbt_merge

ZSTD_OPT='-co COMPRESS=ZSTD -co PREDICTOR=2 -co ZSTD_LEVEL=1 '
TILE_OPT='-co TILED=YES -co blockXsize=1024 -co blockYsize=1024 '
PARAL_OPT='-co NUM_THREADS=ALL_CPUS -multi -wo NUM_THREADS=ALL_CPUS ' # <- || compression, warp and compute
EXTRA_OPT='-co BIGTIFF=YES -overwrite '

In [3]:
alpw_dem_path = os.path.realpath('alps/slopes-Lausanne-Jouques-Sanremo-Zermatt.tif')
#CMAPDIR = '~/code/eddy-geek/TIL/geo/data'

# Mont-Blanc

The area covered is 1.5 z10 tiles around Mont-Blanc, from Morillon to Lavachey
Covers IGN 20m / Aoste 1m / Swisstopo 2m


In [4]:
!mkdir -p montblancz10
%cd montblancz10
# w, s, e, n = 6.855466, 45.828796, 7.207031, 45.951147

/home/me/code/eddy-geek/eslope/development/sample_tests/montblancz10


In [5]:
#other common extents I used:
# see also TIL/geo/src/useful_extents.sh                                           main ⬆ ✱ ◼
# mbz10 = '6.855466 45.828796 7.207031 45.951147'
# clapier='7.38 44.1 7.44 44.15'
# malinvern='7.163085 44.182203 7.207031 44.213709'
# paradis_z11='6.855466 45.460132 7.382813 45.583291'

## Test compression parameters

As per the results below:
* Float compression halves size
  * ZSTD-L1 provides similar results as DEFLATE-L9 with much better write perf
* Float16 halves size with a very small precision cost (<0.06)
* **'Rounding' to Byte divides size by >5** (because it makes for much better compressibility)
This is why I chose to compress to int, and so, why I need palettes with cutoff points at 0.5° -- so that the rounding does not twists the output.

Note: it's important to do this on a representative sample, which is the case here with both low-precision (=good compressibility) and high-precision DEMs.

In [6]:
from time import time
from contextlib import redirect_stderr, redirect_stdout
DEFLATE_OPT = ' -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 '
ZSTDL_OPT=' -co COMPRESS=ZSTD -co PREDICTOR=2 -co ZSTD_LEVEL=%d '
buff = []
with open(os.devnull, 'w') as fnull:
    with redirect_stderr(fnull), redirect_stdout(fnull):
        for name, opt in {
            'f32-': '',
            'f16-': '-co NBITS=16 ',
            'i16-': '-ot UInt16 ',
            'i8-': '-ot Byte ',
            'f16-deflate': '-co NBITS=16 ' + DEFLATE_OPT,
            'i8-deflate': '-ot Byte ' + DEFLATE_OPT,
            'f32-zstd1': ZSTDL_OPT % 1,
            'f16-zstd1': '-co NBITS=16 ' + ZSTDL_OPT % 1,
            'i16-zstd1': '-ot UInt16 ' + ZSTDL_OPT % 1,
            'i8-zstd1': '-ot Byte ' + ZSTDL_OPT % 1,
            'f32-zstd3': ZSTDL_OPT % 3,
            'f16-zstd3': '-co NBITS=16 ' + ZSTDL_OPT % 3,
            'i8-zstd3': '-ot Byte ' + ZSTDL_OPT % 3,
            'f32-zstd9': ZSTDL_OPT % 9,
            'f16-zstd9': '-co NBITS=16 ' + ZSTDL_OPT % 9,
            'i8-zstd9': '-ot Byte ' + ZSTDL_OPT % 9
        }.items():
            dest = f'slopes-z16-cmp-{name}.tif'
            startt = time()
            cut_extent(src=alpw_dem_path, precision='', extent=bbmontblancz10,
                       default_opt=opt + TILE_OPT + PARAL_OPT + EXTRA_OPT, dest=dest)
            buff += [name.rjust(12), ':  ',
                round(os.path.getsize(dest) / 1024**2), ' MB ;  ' ,
                round(time()-startt, 1), ' seconds\n']
            os.remove(dest)

sys.stdout.writelines(map(str, buff))
# test takes 2min

        f32-:  512 MB ;  3.0 seconds
        f16-:  256 MB ;  3.7 seconds
        i16-:  256 MB ;  2.7 seconds
         i8-:  128 MB ;  2.4 seconds
 f16-deflate:  108 MB ;  19.7 seconds
  i8-deflate:  40 MB ;  13.7 seconds
   f32-zstd1:  231 MB ;  3.8 seconds
   f16-zstd1:  107 MB ;  3.5 seconds
   i16-zstd1:  56 MB ;  2.9 seconds
    i8-zstd1:  40 MB ;  2.5 seconds
   f32-zstd3:  235 MB ;  5.3 seconds
   f16-zstd3:  108 MB ;  4.0 seconds
    i8-zstd3:  43 MB ;  2.8 seconds
   f32-zstd9:  230 MB ;  14.4 seconds
   f16-zstd9:  103 MB ;  9.5 seconds
    i8-zstd9:  39 MB ;  5.6 seconds


## Parallelization test

Here is an archive of the results on my laptop:

```
                                                            :  5.8 seconds
                                   -co NUM_THREADS=ALL_CPUS :  4.8 seconds
                                                     -multi :  3.9 seconds
                                   -wo NUM_THREADS=ALL_CPUS :  3.8 seconds
                            -multi -wo NUM_THREADS=ALL_CPUS :  3.6 seconds
   -co NUM_THREADS=ALL_CPUS -multi -wo NUM_THREADS=ALL_CPUS :  2.9 seconds
```
```
                                                            :  7.5 seconds
                                   -co NUM_THREADS=ALL_CPUS :  7.5 seconds
                                                     -multi :  6.7 seconds
                                   -wo NUM_THREADS=ALL_CPUS :  2.9 seconds
                            -multi -wo NUM_THREADS=ALL_CPUS :  2.6 seconds
   -co NUM_THREADS=ALL_CPUS -multi -wo NUM_THREADS=ALL_CPUS :  2.5 seconds
```

In [7]:
cut_extent(src=alpw_dem_path, dest='slopes-z16.tif', extent=bbmontblancz10)

gdalwarp -ot Byte \
    -co COMPRESS=ZSTD -co PREDICTOR=2 -co ZSTD_LEVEL=1 -co TILED=YES -co blockXsize=1024 -co blockYsize=1024 -co NUM_THREADS=ALL_CPUS -multi -wo NUM_THREADS=ALL_CPUS -co BIGTIFF=YES -overwrite  \
     \
    -t_srs EPSG:3857 -tr 2.388657133911758 -2.388657133911758 -r nearest \
    -te_srs WGS84 -te 6.855466 45.828796 7.207031 45.951147 \
    ~/code/eddy-geek/slope-ign-alti/alps/slopes-Lausanne-Jouques-Sanremo-Zermatt.tif slopes-z16.tif
Creating output file that is 16384P x 8192L.
Processing /home/me/code/eddy-geek/slope-ign-alti/alps/slopes-Lausanne-Jouques-Sanremo-Zermatt.tif [1/1] : 0Using internal nodata values (e.g. 255) for image /home/me/code/eddy-geek/slope-ign-alti/alps/slopes-Lausanne-Jouques-Sanremo-Zermatt.tif.
Copying nodata values from source /home/me/code/eddy-geek/slope-ign-alti/alps/slopes-Lausanne-Jouques-Sanremo-Zermatt.tif to destination slopes-z16.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


In [8]:
pbuff = []
base =  ZSTD_OPT + TILE_OPT + EXTRA_OPT
pcompress = '-co NUM_THREADS=ALL_CPUS '
pwarp = '-multi '
pcompute = '-wo NUM_THREADS=ALL_CPUS '
dest = './tmp.tif'
for fun in (
    lambda xopts: make_ovr(src='./slopes-z16.tif', dest=dest, z=15, default_opt=base + xopts),
    lambda xopts: cut_extent(src=alpw_dem_path, dest=dest, extent=bbmontblancz10, default_opt=base + xopts),
):
    with open(os.devnull, 'w') as fnull:
        with redirect_stderr(fnull), redirect_stdout(fnull):
            for xopts in (
                '',
                pcompress,
                pwarp,
                pcompute,
                pwarp+pcompute,
                pcompress+pwarp+pcompute
            ):
                        if os.path.isfile(dest): os.remove(dest)
                        startt = time()
                        fun(xopts)
                        pbuff += [xopts.rjust(60), ':  ',
                            round(os.path.getsize(dest) / 1024**2), ' MB ;  ' ,
                            round(time()-startt, 1), ' seconds\n']

sys.stdout.writelines(map(str, pbuff))

                                                            :  14 MB ;  5.7 seconds
                                   -co NUM_THREADS=ALL_CPUS :  14 MB ;  5.3 seconds
                                                     -multi :  14 MB ;  5.1 seconds
                                   -wo NUM_THREADS=ALL_CPUS :  14 MB ;  2.4 seconds
                            -multi -wo NUM_THREADS=ALL_CPUS :  14 MB ;  2.2 seconds
   -co NUM_THREADS=ALL_CPUS -multi -wo NUM_THREADS=ALL_CPUS :  14 MB ;  1.9 seconds
                                                            :  40 MB ;  4.0 seconds
                                   -co NUM_THREADS=ALL_CPUS :  40 MB ;  3.5 seconds
                                                     -multi :  40 MB ;  2.8 seconds
                                   -wo NUM_THREADS=ALL_CPUS :  40 MB ;  2.8 seconds
                            -multi -wo NUM_THREADS=ALL_CPUS :  40 MB ;  2.5 seconds
   -co NUM_THREADS=ALL_CPUS -multi -wo NUM_THREADS=ALL_CPUS :  40 MB ;  2.1 

## Test overview palette

In [9]:
make_ovr(src='slopes-z16.tif', z=15)



gdalwarp -r q3 -tr 4.777314267823516 -4.777314267823516 \
  -co COMPRESS=ZSTD -co PREDICTOR=2 -co ZSTD_LEVEL=1 -co TILED=YES -co blockXsize=1024 -co blockYsize=1024 -co NUM_THREADS=ALL_CPUS -multi -wo NUM_THREADS=ALL_CPUS -co BIGTIFF=YES -overwrite  \
   \
  slopes-z16.tif slopes-z15.tif
Creating output file that is 8192P x 4096L.
Processing slopes-z16.tif [1/1] : 0Using internal nodata values (e.g. 255) for image slopes-z16.tif.
Copying nodata values from source slopes-z16.tif to destination slopes-z15.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


'slopes-z15.tif'

In [10]:
make_ovr(src='slopes-z16.tif', z=12)
slope_mbt('eslo4near', z=14)



gdalwarp -r q3 -tr 38.21851414258813 -38.21851414258813 \
  -co COMPRESS=ZSTD -co PREDICTOR=2 -co ZSTD_LEVEL=1 -co TILED=YES -co blockXsize=1024 -co blockYsize=1024 -co NUM_THREADS=ALL_CPUS -multi -wo NUM_THREADS=ALL_CPUS -co BIGTIFF=YES -overwrite  \
   \
  slopes-z16.tif slopes-z15.tif
Creating output file that is 1024P x 512L.
Processing slopes-z16.tif [1/1] : 0Using internal nodata values (e.g. 255) for image slopes-z16.tif.
Copying nodata values from source slopes-z16.tif to destination slopes-z15.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
sed 's/nv    0   0   0   0/nv  255 255 255 255/g' \
    ~/code/eddy-geek/TIL/geo/data/gdaldem-slope-eslo4near.clr >! /tmp/gdaldem-slope-eslo4near.clr && \
gdaldem color-relief ./slopes-z14.tif /tmp/gdaldem-slope-eslo4near.clr ./eslo4near-z14.mbtiles \
    -nearest_color_entry -co TILE_FORMAT=png8 
0...10...20...30...40...50...60...70...80...90...100 - done.


'./eslo4near-z14.mbtiles'

In [11]:
make_ovr(src='slopes-z16.tif', z=12)
slope_mbt('eslo4near', z=12)


gdalwarp -r q3 -tr 38.21851414258813 -38.21851414258813 \
  -co COMPRESS=ZSTD -co PREDICTOR=2 -co ZSTD_LEVEL=1 -co TILED=YES -co blockXsize=1024 -co blockYsize=1024 -co NUM_THREADS=ALL_CPUS -multi -wo NUM_THREADS=ALL_CPUS -co BIGTIFF=YES -overwrite  \
   \
  slopes-z16.tif slopes-z15.tif
Creating output file that is 1024P x 512L.
Processing slopes-z16.tif [1/1] : 0Using internal nodata values (e.g. 255) for image slopes-z16.tif.
Copying nodata values from source slopes-z16.tif to destination slopes-z15.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
sed 's/nv    0   0   0   0/nv  255 255 255 255/g' \
    ~/code/eddy-geek/TIL/geo/data/gdaldem-slope-eslo4near.clr >! /tmp/gdaldem-slope-eslo4near.clr && \
gdaldem color-relief ./slopes-z12.tif /tmp/gdaldem-slope-eslo4near.clr ./eslo4near-z12.mbtiles \
    -nearest_color_entry -co TILE_FORMAT=png8 
0...10...20...30...40...50...60...70...80...90...100 - done.


'./eslo4near-z12.mbtiles'

In [12]:
# !pip install pymbtiles
# from pymbtiles.ops import extend
# !cp -f eslo13-z16.mbtiles eslo.mbtiles
# extend('eslo4-z14.mbtiles', 'eslo.mbtiles')
# extend('eslo4near-z12.mbtiles', 'eslo.mbtiles')
# with MBtiles('eslo.mbtiles', 'w+') as m:
#     m.meta['...'] = ...

# !rm eslo.mbtiles
# mbt_merge('eslo13near-z16.mbtiles', 'eslo4near-z14.mbtiles', 'eslo4near-z12.mbtiles', dest='eslo.mbtiles')

In [18]:
make_overviews('./slopes-z16.tif') #, reuse=True)

./eslo.mbtiles
      sed 's/nv    0   0   0   0/nv  255 255 255 255/g' \
          ~/code/eddy-geek/TIL/geo/data/gdaldem-slope-eslo13near.clr >! /tmp/gdaldem-slope-eslo13near.clr && \
      gdaldem color-relief ./slopes-z16.tif /tmp/gdaldem-slope-eslo13near.clr ./eslo13near-z16.mbtiles \
          -nearest_color_entry -co TILE_FORMAT=png8 
0...10...20...30...40...50...60...70...80...90...100 - done.
Step 1/3 completed in 13.0 seconds
      gdalwarp -r q3 -tr 4.777314267823516 -4.777314267823516 \
        -co COMPRESS=ZSTD -co PREDICTOR=2 -co ZSTD_LEVEL=1 -co TILED=YES -co blockXsize=1024 -co blockYsize=1024 -co NUM_THREADS=ALL_CPUS -multi -wo NUM_THREADS=ALL_CPUS -co BIGTIFF=YES -overwrite   \
        ./slopes-z16.tif ./slopes-z15.tif
Creating output file that is 8192P x 4096L.
Processing ./slopes-z16.tif [1/1] : 0Using internal nodata values (e.g. 255) for image ./slopes-z16.tif.
Copying nodata values from source ./slopes-z16.tif to destination ./slopes-z15.tif.
...10...20...30...40..

In [ ]:
os.system('gpxsee eslo.mbtiles &')

0

In [ ]:
for row in ((245, 191,   0),  (220,   0, 245),  (77,  77,  77)):
    for i in row:
        print ('%d ' % ((i*2+255)/3), end='')
    print()

248 212 85 
231 85 248 
136 136 136 
